In [120]:
# 操作 browser 的 驅動程式
from selenium import webdriver

# 負責開啟和關閉 Chrome 的套件
from selenium.webdriver.chrome.service import Service

# 自動下載 Chrome Driver 的套件
from webdriver_manager.chrome import ChromeDriverManager

# 啟動瀏覽器工具的選項
from selenium.webdriver.chrome.options import Options

# 例外處理的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待、了解某個元素的狀態，通常與 exptected_conditions 和 By 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，經常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 隨機取得 User-Agent
from fake_useragent import UserAgent
# ua = UserAgent()


from time import sleep
import json
import random
import os
from datetime import datetime
from datetime import timedelta
import re
from pprint import pprint

In [121]:
user_agent_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.79 Safari/537.36 Edge/14.14393",
    "Mozilla/5.0 (Linux; Android 6.0.1; Nexus 6P Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.83 Mobile Safari/537.36"
    "Mozilla/5.0 (iPhone14,3; U; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) Version/10.0 Mobile/19A346 Safari/602.1"
    # Add more user agent strings as needed
]
user_agent = random.choice(user_agent_list)

my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")  # 最大化視窗
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  # 設定為正體中文
my_options.add_argument('blink-settings=imagesEnabled=false')  # 不載入圖
my_options.add_argument(f'--user-agent={user_agent}')  # 隨機 user-angent 
my_options.add_experimental_option(
    "excludeSwitches", ['enable-automation', 'enable-logging'])  # 沒有異常log


driver = webdriver.Chrome(
    options=my_options,
    service=Service(ChromeDriverManager().install()))
    

[WDM] - Downloading: 100%|██████████| 6.78M/6.78M [00:03<00:00, 2.16MB/s]


In [122]:
# 建立下載路徑/資料夾，不存在就新增 (os.getcwd() 會取得當前的程式工作目錄)
folderPath = os.path.join(os.getcwd(), 'crawler_data')
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

#預設下載路徑
my_options.add_experimental_option("prefs", {
  "download.default_directory": folderPath
})

In [123]:
url = 'https://ctee.com.tw/category/news/stocks'
driver.get(url)

In [124]:
alinks = []
results = []

In [125]:
def inputlink():
    links = driver.find_elements(
        By.CSS_SELECTOR, 
        "#content > div > div > div.col-sm-8.content-column > div.listing.listing-blog.listing-blog-5.clearfix > article > div > h2 > a"
    )
    times = driver.find_elements(
        By.CSS_SELECTOR, 
        "#content > div > div > div > div > article > div > div> span > time"
    )
    
    ctime = datetime.now()
    today9am = ctime.replace(hour=9, minute=0, second=0, microsecond=0)
    yesterday9am = today9am - timedelta(days=1)

    for i in range(len(links)):
        article_datetime = times[i].get_attribute('datetime')
        article_date_str = re.split('\+', article_datetime)[0][2:]
        article_date_time = datetime.strptime(article_date_str, '%y-%m-%dT%H:%M:%S')
        if yesterday9am <= article_date_time:
            href = links[i].get_attribute('href')
            alinks.append(href)
        else:
            return False
    return True

In [126]:
def craw(link):
    title = driver.find_element(
            By.CSS_SELECTOR, 'div.post-header.post-tp-1-header > h1 > span').get_attribute("innerText")
    article_datetime = driver.find_element(
            By.CSS_SELECTOR, 'div.post-header.post-tp-1-header > div.post-meta-wrap.clearfix > div > span > time').get_attribute('datetime')
    article_date_str = re.split('\+', article_datetime)[0].replace('T', ' ')

    article_list = driver.find_elements(
            By.CSS_SELECTOR, 'div.entry-content.clearfix.single-post-content > p')
    article = ''
    for a in article_list:
        article += a.get_attribute("innerText")
  
    reporter_l = article.split('延伸閱讀')[0] # exists recommend  
    reporter = reporter_l.split()[-1][:-1]

    if reporter == '': # exists space after name
        reporter = reporter_l.split()[-2] # get the before one
    if len(reporter) > 3: # no name
        try:
            reporter = driver.find_element(
                    By.CSS_SELECTOR, '#content > div > div > div.col-sm-8.content-column > div.single-container > section > div.author-title.heading-typo > a > span').get_attribute("innerText")
            reporter = reporter.split()[-1]
        except:
            reporter = ''
    if reporter == '數位編輯':
        reporter = ''

    result = {
        'news': '工商時報',
        'link': link, 
        'title': title, 
        'time': article_date_str, 
        'reporter': reporter, 
        'article': article
        }
    
    results.append(result)

In [127]:
if __name__ == '__main__':
    page = 2
    while(inputlink()):
        sleep(1)
        driver.get(url+'/page/'+str(page))
        page += 1
    for link in alinks:
        driver.get(link)
        craw(link)
        sleep(2)

    driver.quit()
    
    filename_innerText = 'test_'
    with open(f'{filename_innerText}'+'.json', "w", encoding='utf-8') as file:
        file.write(json.dumps(
            results, ensure_ascii=False, indent=4))

In [128]:
results

[{'news': '工商時報',
  'link': 'https://ctee.com.tw/news/stocks/812267.html',
  'title': '2/21盤中熱門焦點股',
  'time': '2023-02-21 11:42:44',
  'reporter': '郭鴻慧',
  'article': '今（2/21）日盤中熱門焦點股如下，僅供參考：無人機領導廠商雷虎科技（8033）周二（2/21）開高之後欲小不易，早盤竟在開市10分鐘內即強勢攻頂，緊鎖漲停板，在漲停板門外追價買單還逾1萬張，完全無視大盤拉回整理窘境，表現搶眼。據了解，雷虎搶攻國防軍工商機再下一城！雷虎去年10月獲經濟部遴選為軍用商規微型無人機系統整合主導廠商後，20日公告已與陸軍後勤指揮部簽訂軍品契約TD12002P074微型無人機（原型機）採購契約。法人估，最快年底前可接單生產，為未來營運挹注成長新動能，助攻雷虎近期股價強勢上漲。不僅如此，雷虎在發展軍用商規無人機的同時，今年將把雷虎打造成無人機的孵化器，積極整合具有相關技術優勢的廠商，透過轉投資方式進行策略合作，建立國內無人機產業鏈，目前已有四～五家業者洽談中。雷虎去年合併營收10.89億元，年減近8％；前三季稅後純益0.12億元，EPS為0.09元。受惠軍工股題材發酵，雷虎去年股價飆漲近1.5倍，今年1月合併營收8,538萬元、年增20.2％，為連續三個月呈年增走勢。法人預估，雷虎今年在無人機業務將明顯增長。雷虎2月10日公告辦理1億元現金增資案，將發行1萬張股票，目前處於緘默期。（工商時報 曾麗芳、數位編輯）面板產業供應鏈驅動IC景氣觸底，短期股價持續向上修正合理空間，包括聯詠（3034）、瑞鼎（3592）、敦泰（3545）與矽創（8016）等，今多開高趨堅，臉色紅潤，表現不錯。面板驅動IC市場在歷經超過半年的劇烈庫存調整後，陸續傳來好消息，業界傳出，近期HD規格的低階驅動IC庫存正逐步回到正常水位，加上非中國客戶將訂單移轉到台灣驅動IC廠，因此部分驅動IC廠預計在第二季開始加大投片力道。法人預期，聯詠、敦泰、瑞鼎及矽創等面板驅動IC廠將有望受惠。面板驅動IC市場在去年下半年開始步入劇烈庫存調整，不僅影響到驅動IC廠的出貨動能，同時更面臨驅動IC平均單價逐季下滑的窘境，不過在經過超過半年的劇烈庫存調整潮後，市